# Step 1. 安装依赖

In [ ]:
# 如果缺少 yfinance 请先安装
#!pip install yfinance pandas matplotlib

# Step 2. 导入库 & 下载 VXX 历史数据

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import os

if not os.path.exists('vxx_data.csv'):
    vxx = yf.download("VXX", period="2y", interval="1d")
    vxx.columns = vxx.columns.get_level_values(0)
    vxx = vxx[["Close"]].rename(columns={"Close": "close"})
    vxx.to_csv("vxx_data.csv")
else:
    vxx = pd.read_csv("vxx_data.csv", index_col=0, parse_dates=True)

if not os.path.exists('vix_data.csv'):
    vix = yf.download("^VIX", period="2y", interval="1d")
    vix.columns = vix.columns.get_level_values(0)
    vix = vix[["Close"]].rename(columns={"Close": "vix"})
    vix.to_csv("vxx_data.csv")
else:
    vix = pd.read_csv("vix_data.csv", index_col=0, parse_dates=True)

data = pd.concat([vxx, vix], axis=1).dropna()    
data.head()

In [ ]:
data.plot()

In [ ]:
import numpy as np
# 计算对数收益率
data["ret"] = np.log(data["close"] / data["close"].shift(1))

# 20日滚动标准差（未年化）
data["vol20"] = data["ret"].rolling(window=20).std()

# 年化20日波动率
data["vol20_ann"] = data["vol20"] * np.sqrt(252)

data[["close", "vol20_ann"]].tail(10)

# Step 3. 计算均线 & 生成交易信号

In [ ]:
# 计算10日和30日均线
data["ma10"] = data["close"].rolling(10).mean()
data["ma30"] = data["close"].rolling(30).mean()

# 生成信号：-1=做空, 0=空仓
data["signal"] = 0
data.loc[data["ma10"] < data["ma30"], "signal"] = -1  # 短均线在下 → 做空
data.loc[data["ma10"] >= data["ma30"], "signal"] = 0  # 短均线在上 → 空仓

# VIX 和 波动率信号过滤
data.loc[data["vix"] > 25, "signal"] = 0
data.loc[data["vol20_ann"] > 1, "signal"] = 0

# 最后一天平仓
data.at[data.index[-1], "signal"] = 0

# 检查部分信号
data.tail(10)


In [ ]:
data.loc["2024-07-01":"2024-08-30"]

In [ ]:
# 3. 添加止损逻辑，生成最终信号 'final_signal'
in_position = False
entry_price = None
stop_loss_pct = 0.02  # 止损百分比

for i in range(1, len(data)):
    data.at[data.index[i], "signal"] = data["signal"].iloc[i]
    if not in_position:        
        if data["signal"].iloc[i] == -1 and data["signal"].iloc[i - 1] == 0:
            in_position = True
            entry_price = data["close"].iloc[i]
            print('entry:', data.index[i])
    else:
        current_loss_pct = (data["close"].iloc[i] - entry_price) / entry_price
        if data["signal"].iloc[i] == 0 and data["signal"].iloc[i - 1] == -1:
            print('exit:', data.index[i], entry_price, data["close"].iloc[i], round(current_loss_pct,2))
            in_position = False
            entry_price = None
            continue            
        
        #print('loss:', data.index[i], entry_price, data["close"].iloc[i], round(current_loss_pct,2))

        if current_loss_pct >= stop_loss_pct:
            in_position = False
            entry_price = None
            print('loss exit:', data.index[i], round(current_loss_pct,2))
            data.at[data.index[i], "signal"] = 0

In [ ]:
data['signal'].value_counts()

In [ ]:
data.loc["2025-03-28":"2025-04-03"][["close"]].plot(figsize=(12,6))

In [ ]:
ax = data.loc["2024-07-01":"2024-08-30"][["close","vix"]].plot(figsize=(12,6))

# vol20_ann 放在右侧 Y 轴
data.loc["2024-07-01":"2024-08-30"]["vol20_ann"].plot(ax=ax, secondary_y=True, color="red", label="vol20_ann")

# 设置标题和标签
ax.set_ylabel("Price / VIX")
ax.right_ax.set_ylabel("20-day Ann. Volatility")

plt.title("Close, VIX and 20-day Annualized Volatility")
plt.legend(loc="upper left")
ax.right_ax.legend(loc="upper right")

plt.show()


# Step 4. 计算每日收益 & 策略净值

In [ ]:
# VXX 日度收益率
data["ret"] = data["close"].pct_change()

# 策略收益 = 信号 * VXX收益 （注意：做空时信号 = -1）
data["strategy_ret"] = data["signal"].shift(1) * data["ret"]

# 累积净值
data["cum_vxx"] = (1 + data["ret"]).cumprod()
data["cum_strategy"] = (1 + data["strategy_ret"]).cumprod()

data[["cum_vxx", "cum_strategy"]].plot(figsize=(12,6), title="VXX vs Strategy")
plt.show()


# Step 5. 回测绩效指标

In [ ]:
def performance_stats(returns, freq=252):
    cum_return = (1 + returns).prod() - 1
    ann_return = (1 + cum_return) ** (freq / len(returns)) - 1
    ann_vol = returns.std() * (freq ** 0.5)
    sharpe = ann_return / ann_vol if ann_vol != 0 else 0
    drawdown = ( (1 + returns).cumprod() / (1 + returns).cumprod().cummax() - 1).min()
    return {
        "Cumulative Return": cum_return,
        "Annualized Return": ann_return,
        "Annualized Vol": ann_vol,
        "Sharpe Ratio": sharpe,
        "Max Drawdown": drawdown
    }

stats = performance_stats(data["strategy_ret"].dropna())
stats


# Step 6. 可视化交易信号

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(data.index, data["close"], label="VXX Price", color="black", alpha=0.7)
plt.plot(data.index, data["ma10"], label="10-day MA", color="blue", alpha=0.7)
plt.plot(data.index, data["ma30"], label="30-day MA", color="red", alpha=0.7)

# 做空信号点（10日均线下穿30日均线）
short_signals = (data["signal"] == -1) & (data["signal"].shift(1) == 0)
plt.scatter(data.index[short_signals], data["close"][short_signals],
            marker="v", color="red", label="Enter Short", s=80)

# 平仓信号点（10日均线上穿30日均线）
exit_signals = (data["signal"] == 0) & (data["signal"].shift(1) == -1)
plt.scatter(data.index[exit_signals], data["close"][exit_signals],
            marker="^", color="green", label="Exit Short", s=80)

plt.title("VXX Price with Trading Signals (MA Cross Strategy)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()


# Step 7. 输出交易明细

In [ ]:
trades = []

# 遍历信号，找到进场(-1)和平仓(0)
position = 0
entry_date, entry_price = None, None

for date, row in data.iterrows():
    signal = row["signal"]
    price = row["close"]

    # 开仓：从0 -> -1
    if position == 0 and signal == -1:
        position = -1
        entry_date = date
        entry_price = price

    # 平仓：从-1 -> 0
    elif position == -1 and signal == 0:
        position = 0
        exit_date = date
        exit_price = price
        ret = (entry_price - exit_price) / entry_price  # 做空收益率
        trades.append({
            "Entry Date": entry_date,
            "Entry Price": entry_price,
            "Exit Date": exit_date,
            "Exit Price": exit_price,
            "Holding Days": (exit_date - entry_date).days,
            "Return %": round(ret * 100, 2)
        })

# 转为DataFrame查看
trades_df = pd.DataFrame(trades)
trades_df
